# Catalan Number Generator
This notebook will explore how to generate catalan numbers and their respective sets

# First off we need to define some basic terminology used:
## Dyck words
## Catalan Numbers
## Catalan Set
## Balanced Parenthesis

# Our external libraries used are used as follows:
* randint - used to generate a specific random dyckword of a given length by 

In [ ]:
from random import randint
from timeit import default_timer as timer
from struct import unpack

In [ ]:
def randomDyckWord(output, open, close, pairs):
  if open == pairs and close == pairs:
    return output
  else:
    to_be_appended = randint(0, 1)
    if to_be_appended == 1 and open < pairs:
      return randomDyckWord(output+'1', open+1, close, pairs)
    elif to_be_appended == 0 and close < open:
      return randomDyckWord(output+'0', open, close+1, pairs)
    else:
      return randomDyckWord(output, open, close, pairs)

input: empty string (output), integer 0 (open y close), integer which indicates the number of pair of 0 and 1 to use in the permutation (pairs).

funcion: Output all posible permutations (string) that can be made with a given number of pairs of 1 and 0 (pairs). 

In [ ]:
def dyckWords(output, open, close, pairs):
    if open == pairs and close == pairs:
        yield output
    else:
        if open < pairs:
          yield from dyckWords(output+'1', open+1, close, pairs)
        if close < open:
          yield from dyckWords(output+'0', open, close+1, pairs)

In [ ]:
def areParenthesisBalanced(expr): 
    stack = []  
    for char in expr: 
        if char == '1':   
            stack.append(char) 
        else:
            if not stack: 
                return False
            current_char = stack.pop() 
            if current_char == '1': 
                if char != "0": 
                    return False
    if stack: 
        return False
    return True

Hace una llave mas pequeña

In [ ]:
def retrieve_pairs(expr, pairs):
  sub_key = str()
  ones = 0
  zeroes = 0
  for char in expr:
    if char == '1' and ones < pairs:
      sub_key += '1'
      ones += 1
    elif char == '0' and zeroes < ones:
      sub_key += '0'
      zeroes += 1
  return sub_key

In [ ]:
retrieve_pairs("110101101000", 4)

'11010100'

In [ ]:
def stack_encrypt(text, key):
  cypher_text = str()
  text_lst = [str(char) for char in text]
  stack = list()
  for op in key:
    if op == '1':
      stack.append(text_lst.pop(0))
    if op == '0':
      cypher_text += stack.pop()
  return cypher_text

In [ ]:
def stack_decrypt(cypher, key):
  inverted_key = "".join([str(int(not int(i))) for i in key])[::-1]
  cypher_lst = [str(char) for char in cypher][::-1]
  decyphered_text = str()
  stack = list()
  for op in inverted_key:
    if op == '1':
      stack.append(cypher_lst.pop(0))
    if op == '0':
      decyphered_text += stack.pop()
  return decyphered_text[::-1]

In [ ]:
def catalan_encrypt(text, key):
  cypher_text = str()
  half_key_len = int(len(key)/2)
  if half_key_len <= len(text):
    # if key fits inside text
    if len(text) % half_key_len == 0:
      # if key fits perfectly inside text
      for i in range(0, len(text), half_key_len):
        segment = text[i:i+half_key_len]
        cypher_text += stack_encrypt(segment, key)
    else:
      # if key fits inside text but not perfectly
      special_segment_len = len(text) % half_key_len
      special_segment = text[-special_segment_len:]
      for i in range(0, len(text) - special_segment_len, half_key_len):
        segment = text[i:i+half_key_len]
        cypher_text += stack_encrypt(segment, key)
      sub_key = retrieve_pairs(key, special_segment_len)
      special_segment_cypher = stack_encrypt(special_segment, sub_key)
      cypher_text += special_segment_cypher
  elif half_key_len > len(text):
    sub_key = retrieve_pairs(key, len(text))
    cypher_text = stack_encrypt(text, sub_key)
    
  return cypher_text

In [ ]:
def catalan_decrypt(cypher, key):
  decyphered_text = str()
  half_key_len = int(len(key)/2)
  if half_key_len <= len(cypher):
    # if key fits inside text
    if len(cypher) % half_key_len == 0:
      # if key fits perfectly inside text
      for i in range(0, len(cypher), half_key_len):
        segment = cypher[i:i+half_key_len]
        decyphered_text += stack_decrypt(segment, key)
    else:
      # if key fits inside text but not perfectly
      special_segment_len = len(cypher) % half_key_len
      special_segment = cypher[-special_segment_len:]
      for i in range(0, len(cypher) - special_segment_len, half_key_len):
        segment = cypher[i:i+half_key_len]
        decyphered_text += stack_decrypt(segment, key)
      sub_key = retrieve_pairs(key, special_segment_len)
      special_segment_decyphered = stack_decrypt(special_segment, sub_key)
      decyphered_text += special_segment_decyphered
  elif half_key_len > len(cypher):
    sub_key = retrieve_pairs(key, len(cypher))
    decyphered_text = stack_encrypt(cypher, sub_key)
  return decyphered_text

In [ ]:
binary_string = str()
with open("plainText.txt", "rb") as f:
  while (Byte := f.read(1)):
    print("{0:08b}".format(unpack("b", Byte)[0]))

01100001
01100010
01100011
01100100
01100101
01100110
00100000
01101000
01100101
01101100
01101100
01101111
00001010


In [ ]:
key_pairs = int(input("Please enter desired key length: "))
key = randomDyckWord('', 0, 0, key_pairs)
print(key)

Please enter desired key length: 4
10110010


In [ ]:
secret = catalan_encrypt(binary_string, key)
message = catalan_decrypt(secret, key)
print(secret, "\n" , message)

11000001110010001100100111000100110001011100110010000000110000101100010111000110110001101100111100001010 
 01100001011000100110001101100100011001010110011000100000011010000110010101101100011011000110111100001010
